In [ ]:
!pip install -U torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install mmcv-full
!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
!pip install -e .
!pip install Pillow==7.0.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 704.4 MB 1.2 kB/s 
     |████████████████████████████████| 6.6 MB 2.3 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.12.0+cu113
    Uninstalling torchvision-0.12.0+cu113:
      Successfully uninstalled torchvision-0.12.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.5.1+cu101 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.5.1+cu101 which

In [ ]:
import torch, torchvision
import mmdet
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
import pandas as pd
import numpy as np

In [ ]:
%cd ..

/content


In [ ]:
data = pd.read_csv("Ecoline-2021-09_2022-03-10.csv", sep=";")

In [ ]:
data = data.drop(columns=["FractionId"])
data = data.drop(columns=["FractionName"])
data = data.drop(columns=["SeriesName"])
data = data.drop(columns=["AllPhotoSegmentsClassified"])
data = data.drop(columns=["Question"])

In [ ]:
y = data['Points']

In [ ]:
y = np.array(y)

In [ ]:
arr = np.zeros((len(y), 4))

In [ ]:
for i in range(len(y)):
  arr[i][0] = 1700
  arr[i][1] = 1300

In [ ]:
arr

array([[1700., 1300.,    0.,    0.],
       [1700., 1300.,    0.,    0.],
       [1700., 1300.,    0.,    0.],
       ...,
       [1700., 1300.,    0.,    0.],
       [1700., 1300.,    0.,    0.],
       [1700., 1300.,    0.,    0.]])

In [ ]:
def coord(y, arr):
  ss = 0
  for i in range(len(y)):
    z = y[i]
    z = z[2: len(y[i]) - 2]
    z = z.split("},{")
    for j in range(len(z)):
      tmp = z[j].split(",")
      tmp_1 = tmp[0].split(":")
      tmp_2 = tmp[1].split(":")
      x = int(tmp_1[1])
      cord_y = int(tmp_2[1])
      if x > arr[i][2]:
        arr[i][2] = x
      if x < arr[i][0]:
        arr[i][0] = x
      if cord_y > arr[i][3]:
        arr[i][3] = cord_y
      if cord_y < arr[i][1]:
        arr[i][1] = cord_y
      ss += 1
  return arr, ss

In [ ]:
aray, ss = coord(y, arr)

In [ ]:
pic = data['ImageFileName']

In [ ]:
def pictures(pic):
  new = 'jpg'
  for i in range(len(pic)):
    pic[i] = pic[i][:len(pic[i]) - 3] + new
    # pic[i] = pic[i] + new
  return pic

In [ ]:
pic = pictures(pic)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
pic[int(len(pic) * 0.85)]

'a3c4c5316aec432fad050257b445dcce.jpg'

In [ ]:
list(aray[0])
for i in range(len(aray)):
  aray[i][2] = aray[i][2] - aray[i][0]
  aray[i][3] = aray[i][3] - aray[i][1]

In [ ]:
len(pic)

12035

In [ ]:
aray

array([[ 771.,  561.,  197.,  146.],
       [ 913.,  398.,  543.,  256.],
       [ 820.,  451.,  280.,  155.],
       ...,
       [ 127.,  720.,  344.,  343.],
       [ 256.,   84.,  329.,  261.],
       [1014.,    0.,  189.,  247.]])

In [ ]:
%matplotlib inline
import random

import cv2
from matplotlib import pyplot as plt

import albumentations as A

In [ ]:
df1 = pd.DataFrame(pic)
df2 = pd.DataFrame(aray)
df = pd.concat([df1, df2], axis=1)
df = df.rename(columns={0: 'x_min', 1: 'y_min', 2: 'w', 3: 'h'})
df['classes'] = 0
df

,ImageFileName,x_min,y_min,w,h,classes
0,53b812a205144b49ba0d173b4fb49b64.jpg,771.0,561.0,197.0,146.0,0
1,53b812a205144b49ba0d173b4fb49b64.jpg,913.0,398.0,543.0,256.0,0
2,53b812a205144b49ba0d173b4fb49b64.jpg,820.0,451.0,280.0,155.0,0
3,53b812a205144b49ba0d173b4fb49b64.jpg,914.0,668.0,357.0,201.0,0
4,53b812a205144b49ba0d173b4fb49b64.jpg,809.0,768.0,329.0,284.0,0
...,...,...,...,...,...,...
12030,81f99790c0f54011bbdc75203aa2edf7.jpg,329.0,0.0,142.0,49.0,0
12031,81f99790c0f54011bbdc75203aa2edf7.jpg,60.0,4.0,227.0,152.0,0
12032,81f99790c0f54011bbdc75203aa2edf7.jpg,127.0,720.0,344.0,343.0,0
12033,81f99790c0f54011bbdc75203aa2edf7.jpg,256.0,84.0,329.0,261.0,0


In [ ]:
df['classes'] = data['Answer']
d = {'Ничего из вышеперечисленного': 0, 'ПЭТ коричневый': 1, 'ПЭТ прозрачный': 2, 'ПЭТ зеленый': 3, 'NaN': 4, 'ПЭТ масло': 5, 'ПЭТ молочка (белая/прозрачная)': 6,
     'ПЭТ голубой': 7, 'Одноразовая посуда (пластик)': 8, 'Любая бутылка с термоусадкой': 9, 'Бутылка Бытовая химия': 10, 'Черные металлы': 11, 'Пленка (пакеты/стретч) белая/прозрачная': 12, 'Тэтрапак': 13, 'ПЭТ бирюзовый': 14, 
     'Пленка (пакеты/стретч) цветная': 15, 'ПЭТ остальные цвета': 16, 'Алюминий': 17}
df = df.replace({'classes' : d})
df

,ImageFileName,x_min,y_min,w,h,classes
0,53b812a205144b49ba0d173b4fb49b64.jpg,771.0,561.0,197.0,146.0,0.0
1,53b812a205144b49ba0d173b4fb49b64.jpg,913.0,398.0,543.0,256.0,1.0
2,53b812a205144b49ba0d173b4fb49b64.jpg,820.0,451.0,280.0,155.0,2.0
3,53b812a205144b49ba0d173b4fb49b64.jpg,914.0,668.0,357.0,201.0,3.0
4,53b812a205144b49ba0d173b4fb49b64.jpg,809.0,768.0,329.0,284.0,2.0
...,...,...,...,...,...,...
12030,81f99790c0f54011bbdc75203aa2edf7.jpg,329.0,0.0,142.0,49.0,2.0
12031,81f99790c0f54011bbdc75203aa2edf7.jpg,60.0,4.0,227.0,152.0,0.0
12032,81f99790c0f54011bbdc75203aa2edf7.jpg,127.0,720.0,344.0,343.0,NaN
12033,81f99790c0f54011bbdc75203aa2edf7.jpg,256.0,84.0,329.0,261.0,0.0


In [ ]:
df = df[~np.isnan(df['classes'])]
df

,ImageFileName,x_min,y_min,w,h,classes
0,53b812a205144b49ba0d173b4fb49b64.jpg,771.0,561.0,197.0,146.0,0.0
1,53b812a205144b49ba0d173b4fb49b64.jpg,913.0,398.0,543.0,256.0,1.0
2,53b812a205144b49ba0d173b4fb49b64.jpg,820.0,451.0,280.0,155.0,2.0
3,53b812a205144b49ba0d173b4fb49b64.jpg,914.0,668.0,357.0,201.0,3.0
4,53b812a205144b49ba0d173b4fb49b64.jpg,809.0,768.0,329.0,284.0,2.0
...,...,...,...,...,...,...
12029,81f99790c0f54011bbdc75203aa2edf7.jpg,449.0,640.0,336.0,285.0,10.0
12030,81f99790c0f54011bbdc75203aa2edf7.jpg,329.0,0.0,142.0,49.0,2.0
12031,81f99790c0f54011bbdc75203aa2edf7.jpg,60.0,4.0,227.0,152.0,0.0
12033,81f99790c0f54011bbdc75203aa2edf7.jpg,256.0,84.0,329.0,261.0,0.0


In [ ]:
df = df.reset_index(drop=True)
df

,ImageFileName,x_min,y_min,w,h,classes
0,53b812a205144b49ba0d173b4fb49b64.jpg,771.0,561.0,197.0,146.0,0.0
1,53b812a205144b49ba0d173b4fb49b64.jpg,913.0,398.0,543.0,256.0,1.0
2,53b812a205144b49ba0d173b4fb49b64.jpg,820.0,451.0,280.0,155.0,2.0
3,53b812a205144b49ba0d173b4fb49b64.jpg,914.0,668.0,357.0,201.0,3.0
4,53b812a205144b49ba0d173b4fb49b64.jpg,809.0,768.0,329.0,284.0,2.0
...,...,...,...,...,...,...
9613,81f99790c0f54011bbdc75203aa2edf7.jpg,449.0,640.0,336.0,285.0,10.0
9614,81f99790c0f54011bbdc75203aa2edf7.jpg,329.0,0.0,142.0,49.0,2.0
9615,81f99790c0f54011bbdc75203aa2edf7.jpg,60.0,4.0,227.0,152.0,0.0
9616,81f99790c0f54011bbdc75203aa2edf7.jpg,256.0,84.0,329.0,261.0,0.0


In [ ]:
! unzip ./drive/MyDrive/vvv/ecoline.zip

Archive:  ./drive/MyDrive/vvv/ecoline.zip
  inflating: 001cc7f66f994e2f875490802e49a579.jpg  
  inflating: 001cc7f66f994e2f875490802e49a579.raw  
  inflating: 004f9110946b4c018ea4347d8d569fa5.jpg  
  inflating: 004f9110946b4c018ea4347d8d569fa5.raw  
  inflating: 00551392c98e412cb9facb545ab29278.jpg  
  inflating: 00551392c98e412cb9facb545ab29278.raw  
  inflating: 007a820dca9b442a9d4b27e67634e3cb.jpg  
  inflating: 007a820dca9b442a9d4b27e67634e3cb.raw  
  inflating: 00809adaa33c4c3ebe6e12e542b02843.jpg  
  inflating: 00809adaa33c4c3ebe6e12e542b02843.raw  
  inflating: 00a1a8b2772449d3afaee4ace8921c54.jpg  
  inflating: 00a1a8b2772449d3afaee4ace8921c54.raw  
  inflating: 00b5e587f3d64495aad7c449c33909a6.jpg  
  inflating: 00b5e587f3d64495aad7c449c33909a6.raw  
  inflating: 00c998833af04911af49cbcc600d9a6b.jpg  
  inflating: 00c998833af04911af49cbcc600d9a6b.raw  
  inflating: 00e0927ac3204dc8b1ecaf66dc07a740.jpg  
  inflating: 00e0927ac3204dc8b1ecaf66dc07a740.raw  
  inflating: 0100d0d19

In [ ]:
BOX_COLOR = (255, 0, 0) # Red
TEXT_COLOR = (255, 255, 255) # White


def visualize_bbox(img, bbox, class_name, color=BOX_COLOR, thickness=2):
    """Visualizes a single bounding box on the image"""
    x_min, y_min, w, h = bbox
    x_min, x_max, y_min, y_max = int(x_min), int(x_min + w), int(y_min), int(y_min + h)
   
    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=color, thickness=thickness)
    
    ((text_width, text_height), _) = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, 1.0, 1)    
    cv2.rectangle(img, (x_min, y_min + int(1.0 * text_height)), (x_min + text_width, y_min), BOX_COLOR, -1)
    cv2.putText(
        img,
        text=class_name,
        org=(x_min , y_min + int(0.9 * text_height)),
        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=1.0, 
        color=TEXT_COLOR, 
        lineType=cv2.LINE_AA,
    )
    return img


def visualize(image, bboxes, category_ids, category_id_to_name):
    img = image.copy()
    for bbox, category_id in zip(bboxes, category_ids):
        class_name = category_id_to_name[category_id]
        img = visualize_bbox(img, bbox, class_name)
    # plt.figure(figsize=(12, 12))
    # plt.axis('off')
    # plt.imshow(img)
    return img

In [ ]:
!pip install clearml

     |████████████████████████████████| 794 kB 9.9 MB/s 


In [ ]:
from tqdm import tqdm
import os
import numpy as np
from PIL import Image
# from clearml import Task, Logger

In [ ]:
!clearml-init

ClearML SDK setup process
Configuration file already exists: /root/clearml.conf
Leaving setup, feel free to edit the configuration file.


In [ ]:
def save_pic(df, logger, iteration=0):
  category_id_to_name = {0.0: '0', 1.0: '1', 2.0: '2', 3.0: '3', 5.0: '5', 6.0: '6', 7.0: '7', 8.0: '8', 9.0: '9', 10.0: '10', 11.0: '11', 12.0: '12', 13.0: '13', 14.0: '14', 15.0: '15', 16.0: '16', 17.0: '17'}
  val = 1
  for i in tqdm(range(len(df['ImageFileName'].unique()) - 1)):
    mask = df['ImageFileName'] == df['ImageFileName'].unique()[0]
    df_tmp, df = df[mask], df[~mask]
    df = df.rename(index = lambda x: x - df_tmp.shape[0])
    df_tmp = np.array(df_tmp)
    image = cv2.imread(df_tmp[0][0])
    bboxes = []
    category_ids = []
    for i in range(len(df_tmp)):
      bboxes.append(df_tmp[i][1: 5].tolist())
      category_ids.append(float(df_tmp[i][5: 6]))
    im = visualize(image, bboxes, category_ids, category_id_to_name)
    tit = '/content/tttt/' + str(val) + '.jpg'

    logger.report_image("image", str(val), iteration=iteration, image=im)
    cv2.imwrite(tit, im)
    val += 1
  # return visualize(image, bboxes, category_ids, category_id_to_name)

In [ ]:
df1 = df[8177:].reset_index(drop=True)

In [ ]:
# task = Task.init(project_name="VKR inferens", task_name="test 3")
# logger = task.get_logger()
save_pic(df1, logger)
logger.flush()
task.close()

ClearML Task: overwriting (reusing) task id=6612e65d1d384aeab84dd6123baa933d
ClearML results page: https://app.clear.ml/projects/9b0f6b6cfaa54631b304978a4b226f93/experiments/6612e65d1d384aeab84dd6123baa933d/output/log
2022-05-10 23:03:33,358 - clearml.Task - INFO - No repository found, storing script code instead


  6%|▌         | 9/152 [00:00<00:09, 14.96it/s]

ClearML Monitor: GPU monitoring failed getting GPU reading, switching off GPU monitoring


100%|██████████| 152/152 [00:16<00:00,  9.27it/s]


In [ ]:
%cd mmdetection/

/content/mmdetection


In [ ]:
!mkdir checkpoints

In [ ]:
mask = df['ImageFileName'] == df['ImageFileName'].unique()[0]
df_tmp, df = df[mask], df[~mask]

In [ ]:
df_tmp

,ImageFileName,x_min,y_min,w,h,classes
0,53b812a205144b49ba0d173b4fb49b64.jpg,771.0,561.0,197.0,146.0,0.0
1,53b812a205144b49ba0d173b4fb49b64.jpg,913.0,398.0,543.0,256.0,1.0
2,53b812a205144b49ba0d173b4fb49b64.jpg,820.0,451.0,280.0,155.0,2.0
3,53b812a205144b49ba0d173b4fb49b64.jpg,914.0,668.0,357.0,201.0,3.0
4,53b812a205144b49ba0d173b4fb49b64.jpg,809.0,768.0,329.0,284.0,2.0
5,53b812a205144b49ba0d173b4fb49b64.jpg,358.0,821.0,512.0,293.0,2.0
6,53b812a205144b49ba0d173b4fb49b64.jpg,393.0,1086.0,204.0,128.0,1.0
7,53b812a205144b49ba0d173b4fb49b64.jpg,604.0,1092.0,181.0,108.0,2.0
8,53b812a205144b49ba0d173b4fb49b64.jpg,708.0,1018.0,328.0,197.0,2.0
9,53b812a205144b49ba0d173b4fb49b64.jpg,471.0,440.0,348.0,229.0,0.0


In [ ]:
from mmdet.apis import inference_detector, init_detector, show_result_pyplot

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import cv2
import copy
import os.path as osp
import mmcv
import numpy as np
import mmdet
from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
from mmdet.apis import inference_detector, init_detector, show_result_pyplot
from mmcv import Config
from mmdet.apis import set_random_seed
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets.pipelines.transforms import *
# from clearml import Task

In [ ]:
@DATASETS.register_module()
class KittiTinyDataset(CustomDataset):

    CLASSES = ('0.0', '1.0', '2.0', '3.0', '5.0', '6.0', '7.0', '8.0', '9.0', '10.0', '11.0', '12.0', '13.0', '14.0', '15.0', '16.0', '17.0')

    def load_annotations(self, ann_file):
        cat2label = {k: i for i, k in enumerate(self.CLASSES)}
        image_list = mmcv.list_from_file(self.ann_file)
    
        data_infos = []
        for image_id in image_list:
            filename = f'{self.img_prefix}/{image_id}.jpg'
            image = mmcv.imread(filename)
            height, width = image.shape[:2]
    
            data_info = dict(filename=f'{image_id}.jpg', width=width, height=height)
    
            label_prefix = self.img_prefix.replace('image_2', 'label_2')
            lines = mmcv.list_from_file(osp.join(label_prefix, f'{image_id}.txt'))
    
            content = [line.strip().split(' ') for line in lines]
            bbox_names = [x[0] for x in content]
            bboxes = [[float(info) for info in x[1:5]] for x in content]
    
            gt_bboxes = []
            gt_labels = []
            gt_bboxes_ignore = []
            gt_labels_ignore = []
    
            for bbox_name, bbox in zip(bbox_names, bboxes):
                if bbox_name in cat2label:
                    gt_labels.append(cat2label[bbox_name])
                    gt_bboxes.append(bbox)
                else:
                    gt_labels_ignore.append(-1)
                    gt_bboxes_ignore.append(bbox)

            data_anno = dict(
                bboxes=np.array(gt_bboxes, dtype=np.float32).reshape(-1, 4),
                labels=np.array(gt_labels, dtype=np.int64),
                bboxes_ignore=np.array(gt_bboxes_ignore,
                                       dtype=np.float32).reshape(-1, 4),
                labels_ignore=np.array(gt_labels_ignore, dtype=np.int64))

            data_info.update(ann=data_anno)
            data_infos.append(data_info)

        return data_infos

In [ ]:
def run(df1, model_type, iteration=0):
    if model_type == "ssd":
        cfg = Config.fromfile("/content/mmdetection/configs/ssd/ssd300_coco.py")
    elif model_type == "yolox":
        cfg = Config.fromfile("/content/mmdetection/configs/yolox/yolox_l_8x8_300e_coco.py")
    elif model_type == "faster_rcnn":
        cfg = Config.fromfile("/content/mmdetection/configs/faster_rcnn/faster_rcnn_x101_64x4d_fpn_mstrain_3x_coco.py")

    cfg.dataset_type = "KittiTinyDataset"

    if model_type == "faster_rcnn":
        cfg.model.roi_head.bbox_head.num_classes = 17
    else:
        cfg.model.bbox_head.num_classes = 17

    if model_type == "ssd":
        cfg.load_from = "/content/drive/MyDrive/vvv/ssd_tr_4.pth"
        cfg.work_dir = "/content/mmdetection/ssd"
    elif model_type == "yolox":
        cfg.load_from = "/content/drive/MyDrive/vvv/yolox_tr_2_ep_6.pth"
        cfg.work_dir = "/content/mmdetection/yolox"
    elif model_type == "faster_rcnn":
        cfg.load_from = "/content/drive/MyDrive/vvv/rcnn_tr_5.pth"
        cfg.work_dir = "/content/mmdetection/faster_rcnn"

    model = build_detector( cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
    mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
    model = init_detector(cfg, '/content/drive/MyDrive/vvv/ssd_tr_4.pth', device='cuda:0')
    # model = init_detector(cfg, '/content/drive/MyDrive/vvv/yolox_tr_2_ep_6.pth', device='cuda:0')
    # model = init_detector(cfg, '/content/drive/MyDrive/vvv/rcnn_tr_3_ep_3.pth', device='cuda:0')
    import time
    for i in tqdm(range(len(df1['ImageFileName'].unique()) - 1)):
      start_time = time.time()
      mask = df1['ImageFileName'] == df1['ImageFileName'].unique()[0]
      df_tmp, df1 = df1[mask], df1[~mask]
      df1 = df1.rename(index = lambda x: x - df_tmp.shape[0])
      df_tmp = np.array(df_tmp)
      img = '/content/' + df_tmp[0][0]
      result = inference_detector(model, img)
      # im = show_result_pyplot(model, img, result, score_thr=0.7)
      print("--- %s seconds ---" % (time.time() - start_time))
    result = inference_detector(model, img)
    # im = show_result_pyplot(model, img, result, score_thr=0.7)
    print("--- %s seconds ---" % (time.time() - start_time))

if __name__ == '__main__':
    task = Task.init("VKR", "ssd train_4")
    task = Task.init(project_name="VKR inferens", task_name="faster_rcnn exp3 thr 0.7")
    logger = task.get_logger()
    run(df1,"ssd")
    logger.flush()
    task.close()
    task.close()

load checkpoint from local path: /content/drive/MyDrive/vvv/ssd_tr_4.pth
--- 0.05914115905761719 seconds ---


/content/mmdetection/mmdet/datasets/utils.py:70: UserWarning: "ImageToTensor" pipeline is replaced by "DefaultFormatBundle" for batch inference. It is recommended to manually replace it in the test data pipeline in your config file.
  'data pipeline in your config file.', UserWarning)
